# Testing

In [1]:
import tweepy
import os
import openpyxl
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
consumer_key = os.getenv("CONSUMER_API_KEY")
consumer_secret = os.getenv("CONSUMER_API_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")
bearer_token = os.getenv("BEARER_TOKEN")

In [3]:
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)

# Get My User ID

In [4]:
def get_my_user_id():
    """Retrieve the authenticated user's ID."""
    try:
        user = client.get_me()
        print(user)
        return user.data.id if user.data else None
    except Exception as e:
        print(f"An error occurred while retrieving user ID: {e}")
        return None
    
get_my_user_id()

Response(data=<User id=1917558912703684608 name=Shift username=Shift1646020>, includes={}, errors=[], meta={})


1917558912703684608

# Get Replies on POST

In [5]:
import tweepy

def get_replies_to_tweet(tweet_id, bearer_token):
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
    query = f'conversation_id:{tweet_id} -is:retweet'
    replies = []

    try:
        for response in tweepy.Paginator(
            client.search_recent_tweets,
            query=query,
            tweet_fields=['author_id', 'created_at', 'in_reply_to_user_id'],
            expansions='author_id',
            user_fields=['username'],
            max_results=100
        ):
            if response.data:
                users = {u['id']: u for u in response.includes['users']}
                for tweet in response.data:
                    author = users.get(tweet.author_id)
                    if author:
                        replies.append({
                            'tweet_id': tweet.id,
                            'username': author.username,
                            'created_at': tweet.created_at,
                            'text': tweet.text
                        })
    except Exception as e:
        print(f"An error occurred: {e}")

    return replies

tweet_id = "1916889284298383445" 
replies = get_replies_to_tweet(tweet_id, bearer_token)
# for reply in replies:
#     print(f"Reply ID: {reply['tweet_id']}, @{reply['username']} at {reply['created_at']}: {reply['text']}")
type(replies)

list

In [6]:
replies

[{'tweet_id': 1917382702048895300,
  'username': 'sonjh214136473',
  'created_at': datetime.datetime(2025, 4, 30, 0, 57, 14, tzinfo=datetime.timezone.utc),
  'text': "@MarcStorme @Zeus_arcor @MINDofPepe So you're saying it's not a coin release"},
 {'tweet_id': 1917251650407800893,
  'username': 'MarcStorme',
  'created_at': datetime.datetime(2025, 4, 29, 16, 16, 28, tzinfo=datetime.timezone.utc),
  'text': '@sonjh214136473 @Zeus_arcor @MINDofPepe That’s for the utility, not launch'},
 {'tweet_id': 1917251464277192772,
  'username': 'MarcStorme',
  'created_at': datetime.datetime(2025, 4, 29, 16, 15, 44, tzinfo=datetime.timezone.utc),
  'text': '@AapNootMies7 @Zeus_arcor That’s for the utility'},
 {'tweet_id': 1917142916343939445,
  'username': 'goraragora',
  'created_at': datetime.datetime(2025, 4, 29, 9, 4, 24, tzinfo=datetime.timezone.utc),
  'text': '@MINDofPepe When $MIND burns, the world lights up!'},
 {'tweet_id': 1917138832689967129,
  'username': 'enginakdag',
  'created_at': 

# EXCEL Usernames extractions

In [7]:
import pandas as pd
from urllib.parse import urlparse, unquote

def extract_usernames_from_excel(file_path):
    df = pd.read_excel(file_path)
    
    if 'Profile URL' not in df.columns:
        raise ValueError("The Excel file must contain a 'Profile URL' column.")
    
    usernames = []
    for url in df['Profile URL']:
        if isinstance(url, str):
            parsed_url = urlparse(url)
            path = parsed_url.path
            
            if path.startswith('/search') or path == '/':
                continue
            
            segments = path.strip('/').split('/')
            if segments:
                username = unquote(segments[0])
                usernames.append(username)
    
    return usernames

excel_file = 'data/MIND.xlsx'
usernames = extract_usernames_from_excel(excel_file)
usernames

['flybridge',
 'Vikcyter_2',
 'BRICSinfo',
 'rocketonxyz',
 'Permissionless',
 'TheMoonShow',
 'seedphrase',
 'tether_to',
 'Polymarket',
 'Punk583',
 'R3ACHNTWRK',
 'RobertKennedyJr',
 'oakcurrency',
 'nftfactoryparis',
 '0xPolygonFdn',
 'Solana',
 'BSCNheadlines',
 'jokerace_io',
 'X',
 'JackPosobiec',
 'dexter_cap',
 'divine_economy',
 '1PercentBetterX',
 '0xUnicorn_',
 'deekaymotion',
 'cryptonews',
 'surgence_io',
 'legendarygainz_',
 'Stripe',
 'BackedVC',
 'OthersideMeta',
 'iamneubert',
 'wallet',
 'TrustWallet',
 'BorednHungry',
 '11x_official',
 'xAI',
 'ApeCoin',
 'ton_blockchain',
 'streamhq_xyz',
 'banterboycrypto',
 'NFCSummit',
 'Lacoste',
 'DelGroyp',
 'PayPal',
 'jfx',
 'SOCKETProtocol',
 'bear_brand96',
 'get_wildfire',
 'layer3xyz',
 'cartainc',
 'delegatedotxyz',
 'paulg',
 'IdMintThat',
 'jacqmelinek',
 'HumanEvents',
 'Porsche',
 'IamNomad',
 'HexicanHeartel',
 'JimDaily',
 'DCGco',
 'Layer3FDN',
 'p_ferragu',
 'HendrixxWeb3',
 'BochainCapital',
 'BaycCouncil',
 '

# Tweets I am mentioned

In [8]:
import tweepy

def extract_mentions(bearer_token, username, max_results=100):
    client = tweepy.Client(bearer_token=bearer_token)

    user = client.get_user(username=username)
    user_id = user.data.id

    mentions = client.get_users_mentions(id=user_id, max_results=max_results, expansions='author_id', tweet_fields='created_at')

    mention_details = []

    if mentions.data:
        author_ids = {user.id: user.username for user in mentions.includes['users']}

        for tweet in mentions.data:
            author_username = author_ids.get(tweet.author_id, 'Unknown')
            mention_details.append({
                'username': author_username,
                'tweet_id': tweet.id,
                'text': tweet.text,
                'created_at': tweet.created_at
            })

    return mention_details

username = 'MINDofPepe'
mentions = extract_mentions(bearer_token, username)

for mention in mentions:
    print(f"Username: {mention['username']}")
    print(f"Tweet ID: {mention['tweet_id']}")
    print(f"Text: {mention['text']}")
    print(f"Created At: {mention['created_at']}")
    print('-' * 40)


Username: sonjh214136473
Tweet ID: 1917382702048895300
Text: @MarcStorme @Zeus_arcor @MINDofPepe So you're saying it's not a coin release
Created At: 2025-04-30 00:57:14+00:00
----------------------------------------
Username: VipLiveAlerts
Tweet ID: 1917374857928466486
Text: @ad970900 @MINDofPepe You need to import them. You can see your balance on website when connected too….
Created At: 2025-04-30 00:26:03+00:00
----------------------------------------
Username: VipLiveAlerts
Tweet ID: 1917373932836884803
Text: @MINDofPepe 🔥
Created At: 2025-04-30 00:22:23+00:00
----------------------------------------
Username: _Chukschike
Tweet ID: 1917333314186473613
Text: @MINDofPepe Double power with AI Agent
Created At: 2025-04-29 21:40:59+00:00
----------------------------------------
Username: isoshima_taroh
Tweet ID: 1917288547981353231
Text: @MINDofPepe more&amp;more
Created At: 2025-04-29 18:43:05+00:00
----------------------------------------
Username: deni_liang
Tweet ID: 19172632556571

In [9]:
mentions

[{'username': 'sonjh214136473',
  'tweet_id': 1917382702048895300,
  'text': "@MarcStorme @Zeus_arcor @MINDofPepe So you're saying it's not a coin release",
  'created_at': datetime.datetime(2025, 4, 30, 0, 57, 14, tzinfo=datetime.timezone.utc)},
 {'username': 'VipLiveAlerts',
  'tweet_id': 1917374857928466486,
  'text': '@ad970900 @MINDofPepe You need to import them. You can see your balance on website when connected too….',
  'created_at': datetime.datetime(2025, 4, 30, 0, 26, 3, tzinfo=datetime.timezone.utc)},
 {'username': 'VipLiveAlerts',
  'tweet_id': 1917373932836884803,
  'text': '@MINDofPepe 🔥',
  'created_at': datetime.datetime(2025, 4, 30, 0, 22, 23, tzinfo=datetime.timezone.utc)},
 {'username': '_Chukschike',
  'tweet_id': 1917333314186473613,
  'text': '@MINDofPepe Double power with AI Agent',
  'created_at': datetime.datetime(2025, 4, 29, 21, 40, 59, tzinfo=datetime.timezone.utc)},
 {'username': 'isoshima_taroh',
  'tweet_id': 1917288547981353231,
  'text': '@MINDofPepe m

# Comparison

In [10]:
def filter_replies_by_usernames(replies, target_usernames):
    filtered_replies = []

    for reply in replies:
        if reply['username'] in target_usernames:
            filtered_replies.append({
                'tweet_id': reply['tweet_id'],
                'username': reply['username'],
                'created_at': reply['created_at'],
                'text': reply['text']
            })

    return filtered_replies


filtered_replies = filter_replies_by_usernames(mentions, usernames)

for reply in filtered_replies:
    print(f"Reply ID: {reply['tweet_id']}, @{reply['username']} at {reply['created_at']}: {reply['text']}")


Reply ID: 1916996561999192560, @adekvat276 at 2025-04-28 23:22:51+00:00: @masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.

But I don't think so it's worth to stake now..


In [11]:
filtered_replies

[{'tweet_id': 1916996561999192560,
  'username': 'adekvat276',
  'created_at': datetime.datetime(2025, 4, 28, 23, 22, 51, tzinfo=datetime.timezone.utc),
  'text': "@masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.\n\nBut I don't think so it's worth to stake now.."}]

# Latest Top 3 Posts

In [12]:
def get_latest_top3_tweets(bearer_token, username):
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

    # Get user ID from username
    try:
        user = client.get_user(username=username)
        user_id = user.data.id
    except Exception as e:
        print(f"Failed to get user ID: {e}")
        return []

    # Fetch recent tweets from the user
    try:
        response = client.get_users_tweets(
            id=user_id,
            max_results=10,  # Fetch up to 10 tweets and pick top 3
            tweet_fields=['created_at', 'public_metrics'],
            exclude=['retweets', 'replies']
        )

        tweets = response.data if response.data else []
        top3 = sorted(tweets, key=lambda x: x.created_at, reverse=True)[:3]

        result = []
        for tweet in top3:
            result.append({
                'tweet_id': tweet.id,
                'created_at': tweet.created_at,
                'text': tweet.text,
                'like_count': tweet.public_metrics['like_count'],
                'retweet_count': tweet.public_metrics['retweet_count']
            })

        return result

    except Exception as e:
        print(f"Failed to get tweets: {e}")
        return []

username = "MINDofPepe"

top3_tweets = get_latest_top3_tweets(bearer_token, username)
top3_tweets

[{'tweet_id': 1917230062769983529,
  'created_at': datetime.datetime(2025, 4, 29, 14, 50, 42, tzinfo=datetime.timezone.utc),
  'text': 'Evolve. $MIND https://t.co/4XeTdFqd6T',
  'like_count': 171,
  'retweet_count': 31},
 {'tweet_id': 1916889284298383445,
  'created_at': datetime.datetime(2025, 4, 28, 16, 16, 34, tzinfo=datetime.timezone.utc),
  'text': 'Burn. $MIND https://t.co/cKw3AIMasX',
  'like_count': 187,
  'retweet_count': 29},
 {'tweet_id': 1916527181767053350,
  'created_at': datetime.datetime(2025, 4, 27, 16, 17, 42, tzinfo=datetime.timezone.utc),
  'text': 'Dive into $MIND.\n\nBest Wallet is your best way in.\n\n◉ Track the pre-sale in real-time\n◉ Buy &amp; claim effortlessly\n◉ Get alerts before the masses\n\nThe time draws closer. Stay ahead. 👁\n\nhttps://t.co/Ha0J8BwATp https://t.co/vg8PlcHMNL',
  'like_count': 187,
  'retweet_count': 42}]

# Reply on Tweet

In [13]:
# import tweepy

# def reply_to_tweet(client, tweet_id, reply_text):
#     try:
#         response = client.create_tweet(
#             text=reply_text,
#             in_reply_to_tweet_id=tweet_id
#         )
#         print(f"Replied successfully: https://twitter.com/user/status/{response.data['id']}")
#     except tweepy.TweepyException as e:
#         print(f"An error occurred: {e}")

# reply_to_tweet(client, "1917142916343939445", "burns are where the real games happen")


import tweepy

def reply_to_tweet(client, tweet_id, reply_text):
    try:
        me = client.get_me()
        my_user_id = me.data.id

        query = f'conversation_id:{tweet_id} -is:retweet'
        existing_replies = tweepy.Paginator(
            client.search_recent_tweets,
            query=query,
            tweet_fields=['author_id'],
            max_results=100
        )

        for page in existing_replies:
            if page.data:
                for tweet in page.data:
                    if tweet.author_id == my_user_id:
                        print("Already replied to this tweet.")
                        return
        response = client.create_tweet(
            text=reply_text,
            in_reply_to_tweet_id=tweet_id
        )
        print(f"Replied successfully: https://twitter.com/user/status/{response.data['id']}")

    except tweepy.TweepyException as e:
        print(f"An error occurred: {e}")


reply_to_tweet(client, "1917142916343939445", "burns are where the real games happen")


An error occurred: 403 Forbidden
Your client app is not configured with the appropriate oauth1 app permissions for this endpoint.


# Post Tweets

In [6]:
def post_tweets(client, text, media_paths=None):
    # auth = tweepy.OAuth1UserHandler(
    #     consumer_key,
    #     consumer_secret,
    #     access_token,
    #     access_token_secret
    # )
    # api = tweepy.API(auth)

    media_ids = []
    if media_paths:
        for media_path in media_paths:
            if os.path.isfile(media_path):
                try:
                    media = api.media_upload(media_path)
                    media_ids.append(media.media_id)
                except Exception as e:
                    print(f"Error uploading {media_path}: {e}")
            else:
                print(f"File not found: {media_path}")

    try:
        if media_ids:
            response = client.create_tweet(
                text=text,
                media_ids=media_ids,
                user_auth=True
            )
        else:
            response = client.create_tweet(
                text=text,
                user_auth=True
            )
        print(f"Tweet posted successfully: https://twitter.com/user/status/{response.data['id']}")
        return response
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

In [8]:
text = "UK government draft rules target crypto exchanges & stablecoins 🚦\n\nUK regulators are stepping up oversight with new statutory instruments creating regulated activities like crypto asset trading exchanges and stablecoin issuance. #CryptoRegulation #UKGovTech"

In [9]:
post_tweets(client, text)

An error occurred while posting the tweet: 403 Forbidden
Your client app is not configured with the appropriate oauth1 app permissions for this endpoint.


# Replies Filter based on time

In [15]:
from datetime import datetime, timedelta, timezone

def filter_recent_replies(replies, hours=24):
    now = datetime.now(timezone.utc)
    cutoff = now - timedelta(hours=hours)

    recent_replies = sorted(
        [reply for reply in replies if reply['created_at'] >= cutoff],
        key=lambda r: r['created_at']
    )

    return recent_replies

In [16]:
filter_recent_replies(filtered_replies, hours=24)

[]

In [17]:
filtered_replies

[{'tweet_id': 1916996561999192560,
  'username': 'adekvat276',
  'created_at': datetime.datetime(2025, 4, 28, 23, 22, 51, tzinfo=datetime.timezone.utc),
  'text': "@masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.\n\nBut I don't think so it's worth to stake now.."}]

# Already Commented Check

In [18]:
import tweepy
from datetime import datetime, timezone

def filter_unreplied_tweets(tweets, my_username, bearer_token):
    client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
    unreplied = []

    for tweet in tweets:
        tweet_id = tweet['tweet_id']
        query = f'conversation_id:{tweet_id} -is:retweet'

        try:
            found_reply = False
            for response in tweepy.Paginator(
                client.search_recent_tweets,
                query=query,
                tweet_fields=['author_id', 'created_at'],
                expansions='author_id',
                user_fields=['username'],
                max_results=100
            ):
                if response.data:
                    users = {u['id']: u for u in response.includes['users']}
                    for reply in response.data:
                        author = users.get(reply.author_id)
                        if author and author.username.lower() == my_username.lower():
                            found_reply = True
                            break
                if found_reply:
                    break

            if not found_reply:
                unreplied.append(tweet)

        except Exception as e:
            print(f"Error checking tweet {tweet_id}: {e}")

    return unreplied


In [19]:
filter_unreplied_tweets(filtered_replies, "MINDofPepe", bearer_token )

[{'tweet_id': 1916996561999192560,
  'username': 'adekvat276',
  'created_at': datetime.datetime(2025, 4, 28, 23, 22, 51, tzinfo=datetime.timezone.utc),
  'text': "@masami001001 @MINDofPepe Connect your wallet to their original website. I bought it at around 1400% at January 14. Now I have doubled the amount I bought. Because the % is changing often you get exponentially less and less reward.\n\nBut I don't think so it's worth to stake now.."}]

In [1]:
import tweepy
import os
from dotenv import load_dotenv

load_dotenv()

consumer_key = os.getenv("CONSUMER_API_KEY")
consumer_secret = os.getenv("CONSUMER_API_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

# Set up OAuth1 authentication
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth)

def post_tweet_with_media(text, media_paths=None):
    media_ids = []
    if media_paths:
        for media_path in media_paths:
            if os.path.isfile(media_path):
                try:
                    media = api.media_upload(media_path)
                    media_ids.append(media.media_id)
                except Exception as e:
                    print(f"Error uploading {media_path}: {e}")
            else:
                print(f"File not found: {media_path}")

    try:
        if media_ids:
            tweet = api.update_status(status=text, media_ids=media_ids)
        else:
            tweet = api.update_status(status=text)
        print(f"Tweet posted successfully: https://twitter.com/user/status/{tweet.id}")
        return tweet
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

# Example usage
post_tweet_with_media("Hello World!")


An error occurred while posting the tweet: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product


In [2]:
import tweepy
import os
from dotenv import load_dotenv

load_dotenv()

bearer_token = os.getenv("BEARER_TOKEN")

client = tweepy.Client(bearer_token=bearer_token)

def post_tweet(text):
    try:
        tweet = client.create_tweet(text=text, user_auth=False)
        print(f"Tweet posted successfully: https://twitter.com/user/status/{tweet.data['id']}")
        return tweet
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

# Example usage
post_tweet("Hello World!")


An error occurred while posting the tweet: 403 Forbidden
Authenticating with OAuth 2.0 Application-Only is forbidden for this endpoint.  Supported authentication types are [OAuth 1.0a User Context, OAuth 2.0 User Context].


In [ ]:
import tweepy
import os
from dotenv import load_dotenv
import requests
from urllib.parse import urlencode

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = os.getenv("REDIRECT_URI")

# Step 1: Generate the authorization URL
auth_url = tweepy.OAuth2UserHandler(client_id=client_id, redirect_uri=redirect_uri)
auth_url = auth_url.get_authorization_url()

print(f"Please go to this URL and authorize the application: {auth_url}")

# Step 2: Get the authorization code from the callback
auth_code = input("Enter the authorization code: ")

# Step 3: Exchange the authorization code for an access token
token_url = "https://api.twitter.com/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {client_id}:{client_secret}",
}
body = {
    "code": auth_code,
    "grant_type": "authorization_code",
    "redirect_uri": redirect_uri,
}
response = requests.post(token_url, headers=headers, data=urlencode(body))
response_data = response.json()
access_token = response_data["access_token"]

# Step 4: Use the access token to authenticate and post a tweet
client = tweepy.Client(bearer_token=access_token)

def post_tweet(text):
    try:
        tweet = client.create_tweet(text=text)
        print(f"Tweet posted successfully: https://twitter.com/user/status/{tweet.data['id']}")
        return tweet
    except Exception as e:
        print(f"An error occurred while posting the tweet: {e}")
        return None

# Example usage
post_tweet("Hello World!")
